This script collocate the data of the MOSAiC campaign's radiosoundings and of the OSI SAF's ice concentration for one radiosounding. 

This scrip is used in a bash script to collocate data for multiple radiosoundings.

### IMPORT

In [167]:
from netCDF4 import Dataset as ncfile
import datetime
import os
from sys import exit
from netCDF4 import num2date
import numpy as np
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import pyproj
import glob

### LIST OF DATES FOR THE DESIRED PERIOD

In [198]:
date_min = '20191001000000'
date_max = '20201001000000'

In [199]:
"""Function that create a list of dates and take the date corresponding to the index giving by the bash script."""
def task_date(date_min, date_max):
    current_date = datetime.datetime.strptime(date_min, '%Y%m%d%H%M%S')
    end_date = datetime.datetime.strptime(date_max, '%Y%m%d%H%M%S')
    list_date = []
    while current_date <= end_date:
        list_date.append(current_date.strftime('%Y%m%d%H%M%S'))
        current_date = current_date + datetime.timedelta(hours = 6)
    return(list_date)

In [200]:
liste_date = task_date(date_min,date_max) #str

### FUNCTION

In [201]:
"""Function that finds the indexes of the closest values between OSISAF and MOSAiC.
   Useful for:
- keep the latitudes, longitudes and times of OSISAF corresponding to MOSAiC
- keep the MOSAiC pressure levels corresponding to desired levels"""
def corresponding_index(osi,mosaic) :
    diff = np.absolute(osi-mosaic)
    indx = diff.argmin()
    return indx

### LOOP TO RETRIEVE COLLOCATED SEA ICE DATA AND WRITE IT IN A FILE, FOR EACH DAY

In [ ]:
path_file = '/lustre/storeB/users/justinec/master_internship/data/SEA_ICE/' #path of file where to save sea ice data

for i in range(0,len(liste_date)) :
    date_task = liste_date[i]
    datetask = datetime.datetime.strptime(liste_date[i], '%Y%m%d%H%M%S') #datetime 
    #print('date task :', datetask)
    
    datemin = datetask - datetime.timedelta(days = 3)
    datemax = datetask + datetime.timedelta(days = 3)
    date_min = datemin.strftime('%Y%m%d')
    date_max = datemax.strftime('%Y%m%d')
    year=date_task[0:4] ; month=date_task[4:6] ; day=date_task[6:8] ; hour=date_task[8:10]

    # ------ EUMETSAT OSISAF ------
    ppidir_osi = '/lustre/storeB/project/copernicus/osisaf/data/reprocessed/ice/conc/v3p0/'
    osi_link  = ppidir_osi + year + '/' + month + '/ice_conc_nh_ease2-250_cdr-v3p0_'+year+month+day+'1200.nc'
    osi = ncfile(osi_link,'r')        #dataset of EUMETSAT OSISAF

    # ------ MOSAIC ------
    ppidir_mosaic = '/lustre/storeB/users/maltem/Arctic/MOSAiC/radiosondes/'+year+'/'+month+'/'
    mosaic_link  = ppidir_mosaic + 'PST-RS-01_2_RS41-GDP_001_'+year+month+day+'T'+hour+'0000_1-000-001.nc'
    if os.path.isfile(mosaic_link) == False:
        mosaic_link  = ppidir_mosaic + 'PST-RS-01_2_RS41-GDP_001_'+year+month+day+'T'+hour+'0000_1-000-002.nc'
    if os.path.isfile(mosaic_link) == False:
        mosaic_link  = ppidir_mosaic + 'PST-RS-01_2_RS41-GDP_001_'+year+month+day+'T'+hour+'0000_1-000-003.nc'
    if os.path.isfile(mosaic_link) == False:
        continue
    mosaic = ncfile(mosaic_link,'r')  #dataset of radiosoundings of MOSAiC
    
    mosaic_lat = mosaic.variables['lat'][:]
    mosaic_lon = mosaic.variables['lon'][:]
    mosaic_pres = mosaic.variables['press'][:]

    desired_levels = list(range(300, 1025, 25)) #from 300 hPa to 1000 hPa by 25 hPa
    #desired_levels=era5_pres #if we want the same levels as ERA5 simply decomment this line

    # Considering a single MOSAiC time because radiosounding lasts 1h30. It's short enough for all radiosounding times to be close to the same OSISAF time [12:00]
    indx_time = corresponding_index(osi_nctimenum,mosaic_nctimenum[0])

    # Indexes of MOSAiC pressure levels closest to the desired pressure levels
    indx_level = []
    for i in range(0,len(desired_levels)) :
        indx_level.append(corresponding_index(mosaic_pres,desired_levels[i]))

    mosaic_lat_collocated = mosaic_lat[indx_level]    #keep mosaic's latitudes of each desired pressure level
    mosaic_lon_collocated = mosaic_lon[indx_level]    #keep mosaic's longitudes of each desired pressure level
    mosaic_time_collocated = mosaic_time[indx_level]  #keep mosaic's times of each desired pressure level

    ### CONVERT LATITUDE AND LONGITUDE OF MOSAIC IN COORDINATES OF PROJECTION OF OSISAF
    
    proj_osi = "+proj=laea +lon_0=0 +datum=WGS84 +ellps=WGS84 +lat_0=90.0"
    crs_osi = pyproj.CRS.from_proj4(proj_osi)
    proj_mosaic = "+proj=latlon"
    crs_mosaic = pyproj.CRS.from_proj4(proj_mosaic)
    transform_mosaic_to_osi = pyproj.Transformer.from_crs(crs_mosaic, crs_osi, always_xy = True)
    xx_osiproj, yy_osiproj = transform_mosaic_to_osi.transform(mosaic_lon_collocated[-1], mosaic_lat_collocated[-1])
    xx_osiproj = xx_osiproj*1e-3
    yy_osiproj = yy_osiproj*1e-3

    ### CORRESPONDING INDEXES OF COORDINATES OF PROJECTION XC AND YC BETWEEN OSISAF AND MOSAIC
    osi_xc = osi.variables['xc'][:]
    osi_yc = osi.variables['yc'][:]
    osi_indx_xc = corresponding_index(osi_xc,xx_osiproj)
    osi_indx_yc = corresponding_index(osi_yc,yy_osiproj)

    ### SEA ICE CONCENTRATION FROM OSISAF
    si_conc_osi = osi.variables['ice_conc'][0][osi_indx_yc][osi_indx_xc]
        
    # ------ CRYOSAT-SMOS ------
    ppidir_cryosat = '/lustre/storeB/users/yuriib/thredds/carra-sice-verif/products/CryoSat-2_SMOS/'
    datemin = datetask - datetime.timedelta(days = 3)
    datemax = datetask + datetime.timedelta(days = 3)
    date_min = datemin.strftime('%Y%m%d')
    date_max = datemax.strftime('%Y%m%d')
    cryosat_month_link = ppidir_cryosat + year + '/' + month
    
    if os.path.exists(cryosat_month_link) == False:
        
        ### SAVE DATA IN A .TXT FILE
        Output_file = path_file + 'Sea_ice_data_from_satellites.txt'
        if os.path.isfile(Output_file) == False:
            output_header = 'date'+'\t'+'sea_ice_concentration_osi[%]'+'\t'+'sea_ice_concentration_cryosat-smos[%]'+'\t'+'sea_ice_thickness[m]'+'\t'+'sea_ice_type[2 young, 3 old]'
            #
            Output = open(Output_file, 'w')
            Output.write(output_header + '\n')
            Output.close()
        #
        output_str = date_task + '\t' + str(si_conc_osi) + '\t' + str(np.nan) + '\t' + str(np.nan) + '\t' + str(np.nan)

        #
        Output = open(Output_file, 'a')
        Output.write(output_str + '\n')
        Output.close()
    else:  
        cryosat_link = ppidir_cryosat + year + '/' + month + '/W_XX-ESA,SMOS_CS2,NH_25KM_EASE2_'+date_min+'_'+date_max+'_r_v204_01_l4sit.nc'
        if os.path.isfile(cryosat_link) == False:
            diff = []
            three_days = datetime.timedelta(days = 3)
            liste = sorted(glob.glob(ppidir_cryosat + year + '/' + month + '/W_XX-ESA,SMOS_CS2,NH_25KM_EASE2_*.nc'))
            for i in range(0,len(liste)) :
                datemin = datetime.datetime.strptime(liste[i][117:125], '%Y%m%d')
                middle = datemin + three_days
                if datetask > middle :
                    diff.append(datetask - middle)
                if datetask < middle :
                    diff.append(middle - datetask)
            #print('diff :', diff)
            if np.min(diff) <= three_days :
                #print('diff min :', np.min(diff))
                ind = np.argmin(diff)
                #print('ind :', ind)
                cryosat_link = liste[ind]
                cryosat = ncfile(cryosat_link,'r')  #dataset of CRYOSAT-SMOS
            else :
                continue
        cryosat = ncfile(cryosat_link,'r')  #dataset of CRYOSAT-SMOS
        #print(cryosat_link)
        
        ### CORRESPONDING INDEXES OF COORDINATES OF PROJECTION XC AND YC BETWEEN CRYOSAT-SMOS AND MOSAIC
        cryosat_xc = cryosat.variables['xc'][:]
        cryosat_yc = cryosat.variables['yc'][:]
        cryosat_indx_xc = corresponding_index(cryosat_xc,xx_osiproj)
        cryosat_indx_yc = corresponding_index(cryosat_yc,yy_osiproj)

        ### SEA ICE CONCENTRATION, THICKNESS AND TYPE FROM CRYOSAT-SMOS
        si_conc_cryosat = cryosat.variables['sea_ice_concentration'][0][cryosat_indx_yc][cryosat_indx_xc]
        si_thickness = cryosat.variables['analysis_sea_ice_thickness'][0][cryosat_indx_yc][cryosat_indx_xc]
        si_type = cryosat.variables['sea_ice_type'][0][cryosat_indx_yc][cryosat_indx_xc]
    
        ### SAVE DATA IN A .TXT FILE
        Output_file = path_file + 'Sea_ice_data_from_satellites.txt'
        if os.path.isfile(Output_file) == False:
            output_header = 'date'+'\t'+'sea_ice_concentration_osi[%]'+'\t'+'sea_ice_concentration_cryosat-smos[%]'+'\t'+'sea_ice_thickness[m]'+'\t'+'sea_ice_type[2 young, 3 old]'
            #
            Output = open(Output_file, 'w')
            Output.write(output_header + '\n')
            Output.close()
        #
        output_str = date_task + '\t' + str(si_conc_osi) + '\t' + str(si_conc_cryosat) + '\t' + str(si_thickness) + '\t' + str(si_type)

        #
        Output = open(Output_file, 'a')
        Output.write(output_str + '\n')
        Output.close()